<a href="https://colab.research.google.com/github/asadali163/NLP-Transformers/blob/main/codebase/NER_intermediate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      

In [2]:
from datasets import load_dataset

In [3]:
data = load_dataset('conll2003')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
data['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

Here we are interested in NER tags

In [6]:
data['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [7]:
label_names = data['train'].features['ner_tags'].feature.names

In [8]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [9]:
from transformers import AutoTokenizer

In [10]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [11]:
data['train'][0]['tokens']
# As you can see it is already split into words

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [12]:
# As our tokens is already split into tokens, so we set this to be true
tokens = tokenizer(data['train'][0]['tokens'], is_split_into_words=True)
tokens

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
type(tokens)

transformers.tokenization_utils_base.BatchEncoding

In [14]:
tokens.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

As you can see the lamb is split into two tokens, which we don't want in this case. So we have to fix this issue

For the above issue, we will expand the label. For example if the label for lamb is 5, the we assing the 5 to both la(5), and ##mb(5)

In [15]:
tokens.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [16]:
begin2inside = {
    1: 2,
    3: 4,
    5: 6,
    7: 8,
}

In [17]:
def align_targets(labels, word_ids):
  # label = [7, 0, 0, 0, 3]
  # word_ids = [None, 0, 0, 1, 2, 3, 4, 4]
  # return [-100, 7, 8, 0, 0, 0, 3, 4]
  aligned_labels = []
  last_word = None
  for word in word_ids:
    if word is None: # This will be special token like CLS, SEP
      label = -100
    elif word != last_word:
      label = labels[word]
    else:
      label = labels[word]

      if label in begin2inside:
        label = begin2inside[labels[word]]
    aligned_labels.append(label)
    last_word = word
  return aligned_labels

In [18]:
labels = data['train'][0]['ner_tags']
word_ids = tokens.word_ids()
# print(data['train'][0]['tokens'])
# print(tokens.tokens())
print(labels)
print(word_ids)
# print(len(data['train'][0]['tokens']), len(labels), len(word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]


In [19]:
print(align_targets(labels, word_ids))

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


As on above code the lamb is splitted, but it is not the actual label. So we want actual label, so we will have a fake input for that below

In [20]:
words = [
    '[CLS]', 'Ger', '##man', 'call', 'to', 'bycott', 'Micro', '##soft', '[SEP]'
]
word_ids = [None, 0, 0, 1, 2, 3, 4, 4] # 0 and 4 repeated because of German and Microsoft
labels = [7, 0, 0, 0, 3]
aligned_targets = align_targets(labels, word_ids)
print(aligned_targets)


[-100, 7, 8, 0, 0, 0, 3, 4]


Next, we will define the tokenize function

In [21]:
def tokenize_fn(batch):
  tokenized_inputs = tokenizer(batch['tokens'], truncation=True, is_split_into_words=True)
  labels_batch = batch['ner_tags']
  aligned_labels_batch = []

  for i, labels in enumerate(labels_batch):
    word_ids = tokenized_inputs.word_ids(i)
    aligned_labels_batch.append(align_targets(labels, word_ids))
  tokenized_inputs['labels'] = aligned_labels_batch
  return tokenized_inputs

In [22]:
# Print column names
data['train'].column_names

['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']

For the new tokenized data, we don't need these column names, so we set that in parameters

In [23]:
tokenized_data = data.map(tokenize_fn, batched=True, remove_columns=data['train'].column_names)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [24]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [25]:
from transformers import DataCollatorForTokenClassification

In [26]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [27]:
tokenized_data['train'][0:2]['labels']

[[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]

In [28]:
batch = data_collator([tokenized_data['train'][i] for i in range(2)])
batch['labels']

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

As you can see it is padded and all the other labels are -100

As earlier, when we do metric evaluation, we simply compute it. But now we have multiple targets per sample. So for that we will use another library called <b>seqeval</b>

In [29]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=edaacca96c5ac09fae4290c1aa4b94eea0322f47fce845db1fdd83c199aebd74
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [30]:
from datasets import load_metric

In [31]:
metric = load_metric('seqeval')

<ipython-input-31-1d148ab27cb7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('seqeval')


The repository for seqeval contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/seqeval.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
#This will accept as input the list of list, as normally we will do prediction for multiple sentences a time. So the below will give error
metric.compute(predictions=[0,0,0], references=[0,1,0])

In [32]:
metric.compute(
    predictions=[
        [0,1,1],
        [0,1,0]
    ],
    references=[
        [0,0,1],
        [0,1,0]
    ]

)

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:5

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.8333333333333334}

As this library is build using conll dataset in mind, so it expect us the tag

In [33]:
metric.compute(
    predictions=[
        ['O', 'O', 'I-ORG', 'B-MISC']
    ],
    references=[
        ['O', 'O', 'B-ORG', 'B-MISC']
    ]
)

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 0.75}

In [34]:
import numpy as np

In [35]:
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  preds = np.argmax(logits, axis=-1)

  # remove -100 from label and predictions
  str_labels = [
      [label_names[t] for t in label if t != -100] for label in labels
  ]

  str_preds = [
      [label_names[p] for p, t in zip(pred, targ) if t != -100] for pred, targ in zip(preds, labels)
  ]

  the_metric = metric.compute(predictions=str_preds, references=str_labels)

  return {
      'precision': the_metric['overall_precision'],
      'recall': the_metric['overall_recall'],
      'f1': the_metric['overall_f1'],
      'accuracy': the_metric['overall_accuracy']
  }

Now we will save our labels as same as expected, we will do below, so we can't get the prediction labels later like LABEL_1, LABEL_2, LABEL_3, etc

In [36]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [37]:
from transformers import AutoModelForTokenClassification

In [38]:
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
from transformers import TrainingArguments

In [40]:
training_args = TrainingArguments(
    'saved_model',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01
)

In [41]:
from transformers import Trainer

In [42]:
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_data['train'],
  eval_dataset=tokenized_data['validation'],
  data_collator=data_collator,
  compute_metrics=compute_metrics,
  tokenizer=tokenizer
)

In [43]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.091900,0.089664,0.872395,0.908953,0.890299,0.975231
2,0.045600,0.075487,0.915802,0.935375,0.925485,0.981530
3,0.028500,0.071608,0.917928,0.939246,0.928464,0.982810


TrainOutput(global_step=5268, training_loss=0.08033489013315605, metrics={'train_runtime': 349.5404, 'train_samples_per_second': 120.51, 'train_steps_per_second': 15.071, 'total_flos': 460431563935266.0, 'train_loss': 0.08033489013315605, 'epoch': 3.0})

In [44]:
from transformers import pipeline

In [46]:
trainer.save_model('my_saved_model')

In [47]:
my_model = pipeline(
    'token-classification',
    model='my_saved_model',
    aggregation_strategy='simple',
    device=0
)

In [48]:
# Let's predict
sen = 'My name is Wolfgang and I live in Berlin'
my_model(sen)

[{'entity_group': 'PER',
  'score': 0.99886453,
  'word': 'Wolfgang',
  'start': 11,
  'end': 19},
 {'entity_group': 'LOC',
  'score': 0.9985403,
  'word': 'Berlin',
  'start': 34,
  'end': 40}]